<a href="https://colab.research.google.com/github/DiploDatos/AprendizajeProfundo/blob/master/4_mlflow_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLflow y un experimento sencillo

## Registro de experimentos con MLflow

A la hora de trabajar con problemas de aprendizaje supervisado en general, y de aprendizaje profundo en particular, es necesario llevar un buen registro de los experimentos realizados. En particular, cuando se hace uso de algún tipo de optimización de hiperparámetros (ya sea búsqueda exhaustiva, búsqueda aleatoria o algo más complejo como optimización bayesiana), es crucial tener registro de que se fue haciendo para tomar las decisiones.

[MLflow](https://mlflow.org/) es una plataforma de código abierto que facilita mucho lleva y cuenta de los experimentos que se están realizando. Se pueden ejecutar experimentos y guardar hiperparámetros y métricas de los mismos.

A continuación vamos a ver un ejemplo muy sencillo de como realizar un experimento utilizando MLflow y el conjunto de datos de IMDB.

Gente! Dato importante que faltó en el repo (en breve lo voy a subir)
Para los que trabajen con word embeddings, fijense que los de Glove no les van a servir, porque el MeliChallenge está en español y los embeddings en Ingles. Para esto hay dos opciones igual de válidas:

Descargar embeddings en español
Entrenar sus propios embeddings

Para la opción 1:
Descarguen el los txt de los SBW (los txt).
import bz2
with bz2.open(pretrained_embeddings_path, "rt") as fh: en su modelo, donde está lo de abrir el GZIP
`model = MyClassifier(embs_path="./data/SBW-vectors-300-min5.txt.bz2", emb_size=300)` cuando instancien los modelos. Los embeddings tienen 300 dims.

Para la opción 2:
self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx)
Con vocab_size el tamaño del vocabulario del corpus del MeliChallenge, emb_size un tamaño de embedding (pueden usar 50, directamente si no quieren ajustar esto) y padding_idx=0.

In [1]:
#CORRER UNA VEZ
# !mkdir ./data
# !curl -L https://cs.famaf.unc.edu.ar/\~ccardellino/resources/diplodatos/glove.6B.50d.txt.gz -o ./data/glove.6B.50d.txt.gz
# !curl -L https://cs.famaf.unc.edu.ar/\~ccardellino/resources/diplodatos/meli-challenge-2019.tar.bz2 -o ./data/meli-challenge-2019.tar.bz2
# !tar jxvf ./data/meli-challenge-2019.tar.bz2 -C ./data/
# !pip install mlflow
#!curl -L https://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.txt.bz2 -o ./SBW-vectors-300-min5.txt.bz2

### Librerías

In [2]:
import gzip
import mlflow
import pandas as pd
import tempfile
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from gensim import corpora
from gensim.parsing import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.metrics import balanced_accuracy_score
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm, trange

import json
#import nltk
#from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize
import bz2

tqdm.pandas()


## Dataset

Nos basaremos en el dataset que creamos para el [notebook 2](https://github.com/DiploDatos/AprendizajeProfundo/blob/master/3_datasets.ipynb), con la diferencia de que pasaremos el dataframe de manera directa (esto es para poder hacer split en train/test).

Recordemos de la [notebook 2](https://github.com/DiploDatos/AprendizajeProfundo/blob/master/3_datasets.ipynb) que la clase abstracta `torch.utils.data.Dataset `es la [clase base](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html?highlight=dataset) para construir un dataset de `PyTorch`. Cualquier dataset personalizado debe heredar de dicha clase e implementar los siguientes métodos:

`__len__`: Para que el comando `len(dataset)` devuelva el tamaño del conjunto de datos.

`__getitem__`: Para soportar indexado de manera que `dataset[i]` devuelva el elemento `i`.

In [3]:
class MeliChallenge(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform
    
    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, item):
        if torch.is_tensor(item):
            item = item.to_list()
        
        item = {
            "data": self.dataset.loc[item, "title"],
            "target": self.dataset.loc[item, "category"]
        }
        
        if self.transform:
            item = self.transform(item)
        
        return item

## Preprocesamiento

En este caso vamos a utilizar un sólo módulo para transformar los datos de IMDB. Este se encargará de preprocesar el texto (i.e. normalizarlo) y transformará las palabras en índices de un diccionario para luego poder pasar una secuencia de palabras para buscar en la matriz de embeddings y así permitir mayor manipulación de los embeddings (en lugar de utilizar embeddings fijos).

Vamos a estar trabajando con la librería [gensim](https://pypi.org/project/gensim/) previamente importada para el procesamiento del lenguaje natural (pueden ver su código open source en este [link](https://github.com/RaRe-Technologies/gensim) ).



In [4]:
class RawDataProcessor:
    def __init__(self, 
                 dataset, 
                 ignore_header=True, 
                 filters=None, 
                 vocab_size=50000):
        if filters:
            self.filters = filters
        else:
            self.filters = [
                lambda s: s.lower(),
                preprocessing.strip_tags,
                preprocessing.strip_punctuation,
                preprocessing.strip_multiple_whitespaces,
                preprocessing.strip_numeric,
                preprocessing.remove_stopwords,
                preprocessing.strip_short,
            ]
        
        # Create dictionary based on all the reviews (with corresponding preprocessing)
        # https://radimrehurek.com/gensim/corpora/dictionary.html
        self.dictionary = corpora.Dictionary(
            dataset["title"].map(self._preprocess_string).tolist()
        )
        # Filter the dictionary with extremos words
        # https://tedboy.github.io/nlps/generated/generated/gensim.corpora.Dictionary.filter_extremes.html?highlight=filter_extrem
        self.dictionary.filter_extremes(no_below=2, no_above=1, keep_n=vocab_size)
        
        # Make the indices continuous after some words have been removed
        # https://tedboy.github.io/nlps/generated/generated/gensim.corpora.Dictionary.compactify.html
        self.dictionary.compactify()
        
        # Add a couple of special tokens
        self.dictionary.patch_with_special_tokens({
            "[PAD]": 0,
            "[UNK]": 1
        })
        self.idx_to_target = sorted(dataset["category"].unique())
        self.target_to_idx = {t: i for i, t in enumerate(self.idx_to_target)}


    def _preprocess_string(self, string):
        # https://radimrehurek.com/gensim/parsing/preprocessing.html#gensim.parsing.preprocessing.preprocess_string:~:text=gensim.parsing.preprocessing.preprocess_string
        return preprocessing.preprocess_string(string, filters=self.filters)

    def _sentence_to_indices(self, sentence):
      # https://radimrehurek.com/gensim/corpora/dictionary.html#:~:text=doc2idx(document,via%20unknown_word_index.
        return self.dictionary.doc2idx(sentence, unknown_word_index=1)
    
    def encode_data(self, data):
        return self._sentence_to_indices(self._preprocess_string(data))
    
    def encode_target(self, target):
        return self.target_to_idx[target]
    
    def __call__(self, item):
        if isinstance(item["data"], str):
            data = self.encode_data(item["data"])
        else:
            data = [self.encode_data(d) for d in item["data"]]
        
        if isinstance(item["target"], str):
            target = self.encode_target(item["target"])
        else:
            target = [self.encode_target(t) for t in item["target"]]
        
        return {
            "data": data,
            "target": target
        }

## Lectura de datos

In [5]:
columns_interesting = ['title','category']

In [6]:
train_dataset = pd.concat([x for x in pd.read_json('./data/meli-challenge-2019/spanish.train.jsonl.gz', lines=True, chunksize=100000)], ignore_index=True)
train_dataset = train_dataset[columns_interesting]
train_dataset.head()

,title,category
0,Casita Muñecas Barbies Pintadas,DOLLHOUSES
1,Neceser Cromado Holográfico,TOILETRY_BAGS
2,Funda Asiento A Medida D20 Chevrolet,CAR_SEAT_COVERS
3,Embrague Ford Focus One 1.8 8v Td (90cv) Desde...,AUTOMOTIVE_CLUTCH_KITS
4,Bateria Panasonic Dmwbcf10 Lumix Dmc-fx60n Dmc...,CAMERA_BATTERIES


### Filtro `train_dataset` por todos los datos con las primeras 5 categorías solamente

In [7]:
cat=train_dataset['category'].head(5)

In [8]:
cat

0                DOLLHOUSES
1             TOILETRY_BAGS
2           CAR_SEAT_COVERS
3    AUTOMOTIVE_CLUTCH_KITS
4          CAMERA_BATTERIES
Name: category, dtype: object

In [9]:
train_dataset=train_dataset[train_dataset['category'].isin(list(cat))]
train_dataset.sample()

,title,category
4435177,Fundas Tela Deportiva Fiat Palio Siena Qubo A...,CAR_SEAT_COVERS


In [10]:
train_dataset.reset_index(drop=True, inplace=True)
train_dataset.tail()

,title,category
55666,Portacosméticos De Viaje Victoria's Secret *10...,TOILETRY_BAGS
55667,Set Laundry Pouch + Portacosmetico Colgante,TOILETRY_BAGS
55668,Victoria´s Secret Set 2 Portacosméticos.,TOILETRY_BAGS
55669,Beauty Case Primicia Ontario - Maxime - 6371560,TOILETRY_BAGS
55670,Bateria Original Camara Xiaomi Yi Extra Repues...,CAMERA_BATTERIES


In [11]:
len(train_dataset)

55671

In [12]:
test_dataset = pd.concat([x for x in pd.read_json('./data/meli-challenge-2019/spanish.test.jsonl.gz', lines=True, chunksize=100000)], ignore_index=True)
test_dataset = test_dataset[columns_interesting]
test_dataset.head()

,title,category
0,Mochilas Maternales Bolsos Bebe Simil Cuero Ma...,DIAPER_BAGS
1,Bolso Maternal/bebe Incluye Cambiador + Correa...,DIAPER_BAGS
2,Mochila Maternal Land + Gancho Envio Gratis-cc,DIAPER_BAGS
3,Bolso Maternal Moderno Con Cambiador Y Correa ...,DIAPER_BAGS
4,Bolso Maternal Moderno Con Cambiador Y Correa ...,DIAPER_BAGS


In [13]:
len(test_dataset)

63680

In [14]:
test_dataset=test_dataset[test_dataset['category'].isin(list(cat))]
test_dataset.sample()

,title,category
18924,Portacosmético Victoria´s Secret Roll On!,TOILETRY_BAGS


In [15]:
test_dataset.reset_index(drop=True, inplace=True)
test_dataset.tail()

,title,category
506,Funda Cubre Asiento Auto Cuero Ecológico Unive...,CAR_SEAT_COVERS
507,Funda Eco Cuero Premium A Medida Berlingo Par...,CAR_SEAT_COVERS
508,Funda Asiento Auto Regal Importada Acolchada ...,CAR_SEAT_COVERS
509,Funda Cubre Asiento Cuerina Cuero Auto Vw Gol ...,CAR_SEAT_COVERS
510,Funda Asiento Tela Volkswagen Fox Suran Amarok...,CAR_SEAT_COVERS


## Validation

In [16]:
validation_dataset = pd.concat([x for x in pd.read_json('./data/meli-challenge-2019/spanish.validation.jsonl.gz', lines=True, chunksize=100000)], ignore_index=True)
validation_dataset = validation_dataset[columns_interesting]
validation_dataset.head()

,title,category
0,Metal Biela Dw10 Hdi 2.0,ENGINE_BEARINGS
1,Repuestos Martillo Rotoprcutor Bosch Gshsce Po...,ELECTRIC_DEMOLITION_HAMMERS
2,Pesca Caña Pejerrey Colony Brava 3m Fibra De V...,FISHING_RODS
3,Porcelanato Abitare Be 20x120 Cm. Ceramica Por...,PORCELAIN_TILES
4,Reconstruction Semi Di Lino Alfaparf Shampoo 1...,HAIR_SHAMPOOS_AND_CONDITIONERS


In [17]:
validation_dataset=validation_dataset[validation_dataset['category'].isin(list(cat))]
validation_dataset.sample()

,title,category
192461,Funda Asiento A Medida 1600 Fiat,CAR_SEAT_COVERS


In [18]:
validation_dataset.reset_index(drop=True, inplace=True)
validation_dataset.tail()

,title,category
13913,Bateria Samsung Sb-l220 Vp-d20 Vp-d20i Vp-d21 ...,CAMERA_BATTERIES
13914,Estuches Litoral,TOILETRY_BAGS
13915,Batería Klic7006 Li-40b Li42b Np-80 En-el10 D...,CAMERA_BATTERIES
13916,Cuteroom 1 : 24diy Handicraft Voz En Miniatura...,DOLLHOUSES
13917,Bateria Sony Np-fv70 Dcr-sx21e / Dcrsx21e / Sx21e,CAMERA_BATTERIES


## Preprocess

In [19]:
preprocess_train = RawDataProcessor(train_dataset)
train_dataset = MeliChallenge(train_dataset, transform=preprocess_train)

In [20]:
preprocess_test = RawDataProcessor(test_dataset)
test_dataset = MeliChallenge(test_dataset, transform=preprocess_test)

In [21]:
preprocess_validation = RawDataProcessor(validation_dataset)
validation_dataset = MeliChallenge(validation_dataset, transform=preprocess_test)

## Collation function

Como en este caso trabajamos con secuencias de palabras (representadas por sus índices en un vocabulario), cuando queremos buscar un *batch* de datos, el `DataLoader` de PyTorch espera que los datos del *batch* tengan la misma dimensión (para poder llevarlos todos a un tensor de dimensión fija). Esto lo podemos lograr mediante el parámetro de `collate_fn`. En particular, esta función se encarga de tomar varios elementos de un `Dataset` y combinarlos de manera que puedan ser devueltos como un tensor de PyTorch. Muchas veces la `collate_fn` que viene por defecto en `DataLoader` sirve (como se vio en el notebook 2), pero este no es el caso. Se define un módulo `PadSequences` que toma un valor mínimo, opcionalmente un valor máximo y un valor de relleno (*pad*) y dada una lista de secuencias, devuelve un tensor con *padding* sobre dichas secuencias.

In [22]:
class PadSequences:
    def __init__(self, pad_value=0, max_length=None, min_length=1):
        assert max_length is None or min_length <= max_length
        self.pad_value = pad_value
        self.max_length = max_length
        self.min_length = min_length

    def __call__(self, items):
        data, target = list(zip(*[(item["data"], item["target"]) for item in items]))
        seq_lengths = [len(d) for d in data]

        if self.max_length:
            max_length = self.max_length
            seq_lengths = [min(self.max_length, l) for l in seq_lengths]
        else:
            max_length = max(self.min_length, max(seq_lengths))

        data = [d[:l] + [self.pad_value] * (max_length - l)
                for d, l in zip(data, seq_lengths)]
            
        return {
            "data": torch.LongTensor(data),
            "target": torch.FloatTensor(target)
        }

## DataLoaders

Ya habiendo definido nuestros conjuntos de datos y nuestra `collation_fn`, podemos definir nuestros `DataLoader`, uno para entrenamiento y otro para evaluación. Ver que la diferencia fundamental está en `shuffle`, no queremos mezclar los valores de evaluación cada vez que evaluamos porque al evaluar mediante *mini-batchs* nos puede generar inconsistencias.

In [23]:
pad_sequences = PadSequences()
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True,
                          collate_fn=pad_sequences, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False,
                         collate_fn=pad_sequences, drop_last=False)
validation_loader = DataLoader(validation_dataset, batch_size=128, shuffle=False,
                         collate_fn=pad_sequences, drop_last=False)

## El modelo de clasificación

Para clasificación utilizaremos un perceptrón multicapa de dos capas ocultas. Claramente este modelo es naive y prácticamente todo lo que está *hardcodeado* (e.g. los tamaños de las capas o la cantidad de capas) podría ser parte de los parámetros del modelo. En particular, tenemos la capa de `Embeddings` que es rellenada con los valores de embeddings preentrenados (los de Glove en este caso).

In [24]:
class MeliClassifier(nn.Module):
    def __init__(self, 
                 pretrained_embeddings_path, 
                 dictionary,
                 vector_size,
                 freeze_embedings):
        super().__init__()
        embeddings_matrix = torch.randn(len(dictionary), vector_size)
        embeddings_matrix[0] = torch.zeros(vector_size)
        with bz2.open(pretrained_embeddings_path, "rt") as fh:
            for line in fh:
                word, vector = line.strip().split(None, 1)
                if word in dictionary.token2id:
                    embeddings_matrix[dictionary.token2id[word]] =\
                        torch.FloatTensor([float(n) for n in vector.split()])
        self.embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
                                                       freeze=freeze_embedings,
                                                       padding_idx=0)
        self.hidden1 = nn.Linear(vector_size, 512)
        self.hidden2 = nn.Linear(512, 256)
        self.hidden3 = nn.Linear(256, 128)
        self.hidden4 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 5)              #632 clases de salida.
        #self.output_layer = nn.Linear(128, 632)              #632 clases de salida.
        self.vector_size = vector_size
    
    def forward(self, x):
        x = self.embeddings(x)
        x = torch.mean(x, dim=1)
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = F.relu(self.hidden3(x))
        x = F.relu(self.hidden4(x))
        x = self.output(x)  # Output Layer
        x = F.softmax(x, dim=1)
        #x = torch.sigmoid(self.output(x))
        #x = torch.nn.Softmax(self.output(x))
        return x

## Experimento de MLflow

Por último, ya tenemos todos los bloques para construir nuestro experimento de MLflow. Anotamos un par de parámetros  (estos pueden ser todos los que se consideren necesarios) y lanzamos a correr nuestro experimento. Cada vez que finaliza un epoch guardamos algunas métricas. Al finalizar todos los epochs corremos algunas métricas extras de evaluación y guardamos algunos datos extra que nos servirán para calcular otras métricas a futuro.

In [25]:
torch.cuda.is_available()

True

Prueba con 80 epocas

In [26]:
mlflow.set_experiment("NaiveMeliChallenge_experiment_5Cat")
EPOCHS=80
with mlflow.start_run():
    mlflow.log_param("model_name", "mlp1_5Cat")
    mlflow.log_param("freeze_embedding", True)
    mlflow.log_params({
        "embedding_size": 300,
        "hidden1_size": 512,
        "hidden2_size": 256,
        "hidden3_size": 128,
        "hidden4_size": 64,
        "Epochs": EPOCHS,
        "optimizer": "optim.SGD_lr=0.01_momentum=0.9",
    })
    model = MeliClassifier("./SBW-vectors-300-min5.txt.bz2", preprocess_train.dictionary, 300, True) # acá reemplazar con
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    loss = nn.CrossEntropyLoss() #nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)#optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    for epoch in trange(EPOCHS):
        model.train()
        running_loss = []
        targets = []
        predictions = []
        for idx, batch in enumerate(tqdm(train_loader)):
            optimizer.zero_grad()
            output = model(batch["data"].cuda())
            loss_value = loss(output, batch["target"].cuda().squeeze().long()) #.view(-1, 1))
            loss_value.backward()
            optimizer.step()
            running_loss.append(loss_value.item())        
            targets.extend(batch["target"].numpy())
            predictions.extend(torch.argmax(output.cpu(), 1).numpy())
        mlflow.log_metric("train_loss", sum(running_loss) / len(running_loss), epoch)
        mlflow.log_metric("train_bas", balanced_accuracy_score(targets, predictions), epoch)
        model.eval()
        running_loss = []
        targets = []
        predictions = []
        for batch in tqdm(validation_loader):
            output = model(batch["data"].cuda())
            running_loss.append(
                loss(output, batch["target"].cuda().squeeze().long()).item() #.view(-1, 1)).item()
            )
            targets.extend(batch["target"].numpy())
            predictions.extend(torch.argmax(output.cpu(), 1).numpy())
        mlflow.log_metric("validation_loss", sum(running_loss) / len(running_loss), epoch)
        #mlflow.log_metric("validation_avp", average_precision_score(targets, predictions), epoch)
        mlflow.log_metric("validation_bas", balanced_accuracy_score(targets, predictions), epoch)
    
    with tempfile.TemporaryDirectory() as tmpdirname:
        targets = []
        predictions = []
        for batch in tqdm(validation_loader):
            output = model(batch["data"].cuda())
            targets.extend(batch["target"].cpu().numpy())
            predictions.extend(output.cpu().squeeze().detach().numpy())
        pd.DataFrame({"prediction": predictions, "target": targets}).to_csv(
            f"{tmpdirname}/predictions.csv.gz", index=False
        )
        mlflow.log_artifact(f"{tmpdirname}/predictions.csv.gz")

2022/11/01 12:43:13 INFO mlflow.tracking.fluent: Experiment with name 'NaiveMeliChallenge_experiment_5Cat' does not exist. Creating a new experiment.


  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/435 [00:00<?, ?it/s]

  0%|          | 0/109 [00:00<?, ?it/s]

  0%|          | 0/435 [00:00<?, ?it/s]

...

  0%|          | 0/435 [00:00<?, ?it/s]

  0%|          | 0/109 [00:00<?, ?it/s]

  0%|          | 0/435 [00:00<?, ?it/s]

  0%|          | 0/109 [00:00<?, ?it/s]

  0%|          | 0/109 [00:00<?, ?it/s]

In [29]:
mlflow.tracking.MlflowClient().list_experiments()
mlflow.search_runs()

/tmp/ipykernel_2313467/802100939.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.list_experiments`` is deprecated. This method will be removed in a future release. Use ``search_experiments()`` instead.
  mlflow.tracking.MlflowClient().list_experiments()


,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.validation_loss,metrics.train_bas,metrics.train_loss,metrics.validation_bas,...,params.hidden2_size,params.hidden1_size,params.Epochs,params.freeze_embedding,params.hidden4_size,params.model_name,tags.mlflow.user,tags.mlflow.source.type,tags.mlflow.source.name,tags.mlflow.runName
0,9d97b6dcf63047c293d0c020b592399a,2,FINISHED,file:///users/cdagatti/Deep-Learning/mlruns/2/...,2022-11-01 15:43:14.445000+00:00,2022-11-01 16:03:28.245000+00:00,1.684378,0.700798,1.141588,0.19202,...,256,512,80,True,64,mlp1_5Cat,cdagatti,LOCAL,/users/cdagatti/miniconda3/envs/deeplearning/l...,amusing-snipe-392


In [30]:
from sklearn.metrics import classification_report
y_true = []
y_pred = []
with torch.no_grad():
    for data in tqdm(test_loader):
        inputs, labels = data
        outputs = model(data['data'].cuda())
        _, predicted = torch.max(outputs.data, 1)
        y_true.extend(data['target'].numpy())
        y_pred.extend(predicted.cpu().numpy())
        
#print ('y_true: ', y_true)
#print ('y_pred: ', y_pred)
#print ('y_true: ', y_true)
print(classification_report(y_true, y_pred, target_names=cat))

  0%|          | 0/4 [00:00<?, ?it/s]

                        precision    recall  f1-score   support

            DOLLHOUSES       0.00      0.00      0.00       101
         TOILETRY_BAGS       0.27      0.35      0.31       102
       CAR_SEAT_COVERS       0.09      0.08      0.09       100
AUTOMOTIVE_CLUTCH_KITS       0.00      0.00      0.00       100
      CAMERA_BATTERIES       0.22      0.41      0.29       108

              accuracy                           0.17       511
             macro avg       0.12      0.17      0.14       511
          weighted avg       0.12      0.17      0.14       511



/users/cdagatti/miniconda3/envs/deeplearning/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/users/cdagatti/miniconda3/envs/deeplearning/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/users/cdagatti/miniconda3/envs/deeplearning/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

Prueba con 200 epochs

In [31]:
mlflow.set_experiment("NaiveMeliChallenge_experiment_5Cat")
EPOCHS=200
with mlflow.start_run():
    mlflow.log_param("model_name", "mlp1_5Cat")
    mlflow.log_param("freeze_embedding", True)
    mlflow.log_params({
        "embedding_size": 300,
        "hidden1_size": 512,
        "hidden2_size": 256,
        "hidden3_size": 128,
        "hidden4_size": 64,
        "Epochs": EPOCHS,
        "optimizer": "optim.SGD_lr=0.01_momentum=0.9",
    })
    model = MeliClassifier("./SBW-vectors-300-min5.txt.bz2", preprocess_train.dictionary, 300, True) # acá reemplazar con
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    loss = nn.CrossEntropyLoss() #nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)#optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    for epoch in trange(EPOCHS):
        model.train()
        running_loss = []
        targets = []
        predictions = []
        for idx, batch in enumerate(tqdm(train_loader)):
            optimizer.zero_grad()
            output = model(batch["data"].cuda())
            loss_value = loss(output, batch["target"].cuda().squeeze().long()) #.view(-1, 1))
            loss_value.backward()
            optimizer.step()
            running_loss.append(loss_value.item())        
            targets.extend(batch["target"].numpy())
            predictions.extend(torch.argmax(output.cpu(), 1).numpy())
        mlflow.log_metric("train_loss", sum(running_loss) / len(running_loss), epoch)
        mlflow.log_metric("train_bas", balanced_accuracy_score(targets, predictions), epoch)
        model.eval()
        running_loss = []
        targets = []
        predictions = []
        for batch in tqdm(validation_loader):
            output = model(batch["data"].cuda())
            running_loss.append(
                loss(output, batch["target"].cuda().squeeze().long()).item() #.view(-1, 1)).item()
            )
            targets.extend(batch["target"].numpy())
            predictions.extend(torch.argmax(output.cpu(), 1).numpy())
        mlflow.log_metric("validation_loss", sum(running_loss) / len(running_loss), epoch)
        #mlflow.log_metric("validation_avp", average_precision_score(targets, predictions), epoch)
        mlflow.log_metric("validation_bas", balanced_accuracy_score(targets, predictions), epoch)
    
    with tempfile.TemporaryDirectory() as tmpdirname:
        targets = []
        predictions = []
        for batch in tqdm(validation_loader):
            output = model(batch["data"].cuda())
            targets.extend(batch["target"].cpu().numpy())
            predictions.extend(output.cpu().squeeze().detach().numpy())
        pd.DataFrame({"prediction": predictions, "target": targets}).to_csv(
            f"{tmpdirname}/predictions.csv.gz", index=False
        )
        mlflow.log_artifact(f"{tmpdirname}/predictions.csv.gz")

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/435 [00:00<?, ?it/s]

  0%|          | 0/109 [00:00<?, ?it/s]

  0%|          | 0/435 [00:00<?, ?it/s]

  0%|          | 0/109 [00:00<?, ?it/s]

...

  0%|          | 0/435 [00:00<?, ?it/s]

  0%|          | 0/109 [00:00<?, ?it/s]

  0%|          | 0/435 [00:00<?, ?it/s]

  0%|          | 0/109 [00:00<?, ?it/s]

  0%|          | 0/109 [00:00<?, ?it/s]

In [33]:
print(model)

MeliClassifier(
  (embeddings): Embedding(7188, 300, padding_idx=0)
  (hidden1): Linear(in_features=300, out_features=512, bias=True)
  (hidden2): Linear(in_features=512, out_features=256, bias=True)
  (hidden3): Linear(in_features=256, out_features=128, bias=True)
  (hidden4): Linear(in_features=128, out_features=64, bias=True)
  (output): Linear(in_features=64, out_features=5, bias=True)
)


In [34]:
mlflow.tracking.MlflowClient().list_experiments()
mlflow.search_runs()

/tmp/ipykernel_2313467/802100939.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.list_experiments`` is deprecated. This method will be removed in a future release. Use ``search_experiments()`` instead.
  mlflow.tracking.MlflowClient().list_experiments()


,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.validation_loss,metrics.train_bas,metrics.train_loss,metrics.validation_bas,...,params.optimizer,params.hidden1_size,params.Epochs,params.freeze_embedding,params.hidden4_size,params.model_name,tags.mlflow.user,tags.mlflow.source.type,tags.mlflow.source.name,tags.mlflow.runName
0,b90ff4377ee44270a8f846a88db175eb,2,FINISHED,file:///users/cdagatti/Deep-Learning/mlruns/2/...,2022-11-01 16:05:24.680000+00:00,2022-11-01 16:37:12.882000+00:00,1.709683,0.729359,1.103314,0.174919,...,optim.SGD_lr=0.01_momentum=0.9,512,200,True,64,mlp1_5Cat,cdagatti,LOCAL,/users/cdagatti/miniconda3/envs/deeplearning/l...,gaudy-dolphin-859
1,9d97b6dcf63047c293d0c020b592399a,2,FINISHED,file:///users/cdagatti/Deep-Learning/mlruns/2/...,2022-11-01 15:43:14.445000+00:00,2022-11-01 16:03:28.245000+00:00,1.684378,0.700798,1.141588,0.192020,...,optim.SGD_lr=0.01_momentum=0.9,512,80,True,64,mlp1_5Cat,cdagatti,LOCAL,/users/cdagatti/miniconda3/envs/deeplearning/l...,amusing-snipe-392


In [35]:
from sklearn.metrics import classification_report
y_true = []
y_pred = []
with torch.no_grad():
    for data in tqdm(validation_loader):
        inputs, labels = data
        outputs = model(data['data'].cuda())
        _, predicted = torch.max(outputs.data, 1)
        y_true.extend(data['target'].numpy())
        y_pred.extend(predicted.cpu().numpy())
        
print(classification_report(y_true, y_pred, target_names=cat))

  0%|          | 0/109 [00:00<?, ?it/s]

                        precision    recall  f1-score   support

            DOLLHOUSES       0.03      0.01      0.01      3371
         TOILETRY_BAGS       0.23      0.55      0.33      2926
       CAR_SEAT_COVERS       0.15      0.09      0.12      3044
AUTOMOTIVE_CLUTCH_KITS       0.00      0.00      0.00      1793
      CAMERA_BATTERIES       0.14      0.22      0.17      2784

              accuracy                           0.18     13918
             macro avg       0.11      0.17      0.13     13918
          weighted avg       0.12      0.18      0.13     13918



/users/cdagatti/miniconda3/envs/deeplearning/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/users/cdagatti/miniconda3/envs/deeplearning/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/users/cdagatti/miniconda3/envs/deeplearning/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

## Modificar y probar
cantidad de capas ocultas 

epochs

learning rate

funciones de activación

quitando / agregando capa de softmax 

In [32]:
# Para analizar con el mismo modelo distintos experimentos

# wd_experiments
# data = []
# for experiment in wd_experiments:
#     wd = experiment['weight_decay']
#     for epoch, train_loss in experiment['train_loss']:
#         data.append((epoch, wd, train_loss, 'train'))
#     for epoch, val_loss in experiment['val_loss']:
#         data.append((epoch, wd, val_loss, 'val'))
# data = pandas.DataFrame(data=data, columns=['epoch', 'weight_decay', 'loss', 'dataset'])
# data[:3]

In [ ]:
# Grafico de perdida de loss en epocas
# data = []
# for epoch, (train_loss, val_loss) in enumerate(zip(
#         experiment['train_loss'], experiment['validation_loss'])):
#     data.append((epoch, 'train', train_loss[1]))
#     data.append((epoch, 'val', validation_loss[1]))
# data = pandas.DataFrame(data=data, columns=['epoch', 'dataset', 'loss'])
# seaborn.pointplot(
#     data=data, x='epoch', y='loss', hue='dataset'
# )